In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
nlp=spacy.load('en')

In [ ]:
beaches=pd.read_csv("https://s3.us-east-2.amazonaws.com/datafaculty/beaches.csv")

In [ ]:
beaches.head()

In [ ]:
## In each sentence find out the nouns
## Find out the nouns most similar to the aspects
## 
stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
            "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
            "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", 
            "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", 
            "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", 
            "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", 
            "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", 
            "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", 
            "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
            "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

def get_nouns(x):
    doc=nlp(x) ## Tokenize and extract grammatical components
    doc=[i.text for i in doc if i.text not in stop_words and i.pos_=="NOUN"] ## Remove common words and retain only nouns
    doc=list(map(lambda i: i.lower(),doc)) ## Normalize text to lower case
    return ",".join(doc)

In [ ]:
%%time
beaches['nouns']=beaches['Review_text'].map(get_nouns)

In [ ]:
beaches.head()

In [ ]:
import gensim
from gensim.models import KeyedVectors

In [ ]:
%%time
# local_path="/Users/gunnvantsaini/Data/Work/ML Course/Module 5 Neural Networks/Data/glove.6B.100d.w2vformat.txt"
w2vec=KeyedVectors.load_word2vec_format("glove.6B.300d.w2vformat.txt")
#w2vec=KeyedVectors.load_word2vec_format(local_path)
# if on local system use the path the glove vector file

In [ ]:
def get_similar_beauty(x):
    x=x.split(",")
    rel_aspects=[]
    for noun in x:
        try:
            if w2vec.similarity(noun,"beauty")>0.50:
                rel_aspects.append(noun)
            else:
                continue
        except:
            continue
    if len(rel_aspects)==0:
        rel_aspects.append("None")
    return ",".join(rel_aspects)
        

In [ ]:
beaches['beauty_aspects']=beaches['nouns'].map(get_similar_beauty)

In [ ]:
beaches